# Kaggle
## 개요
[Kaggle](https://www.kaggle.com/)은 내가 만든 머신러닝 모델을 실제 데이터에 적용하여 테스트 할 수 있는 사이트이다. 주어진 문제에 대해 얼마나 좋은 성능을 내는 모델을 개발해 정답에 가까운 예측을 하는지 여부가 Leaderboard에 기록되기도 한다. 많은 Kaggle 경험 자체가 포트폴리오가 될 수 있는 매우 강력한 서비스이다.

사이트를 들어가면 여러 사람들이 공유한 학습 방법과 데이터 전처리 방법이 있고 커뮤니티가 잘 활성화 되어있다. 이를 통해 많은 것을 배울 수도 있다.

실전에서는 모델을 잘 다루는 것 만큼 중요한 것이 데이터를 정제하는 **데이터 전처리** 작업이다. 실습을 통해 알아보자.

## 실습
입문자들이 하기 가장 좋은 Titanic 문제를 이용하여 Kaggle 사용법을 익혀보자. [Titanic](https://www.kaggle.com/c/titanic/data?select=gender_submission.csv)

먼저 링크에 접속하여 문제를 읽고 Download All 버튼을 눌러 데이터를 다운받는다. 다운 받은 후 데이터에 대한 설명을 읽어보자. 배에 올라탄 사람들에 대한 여러 정보가 담겨있는 데이터임을 알 수 있다.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from google.colab import files
import io

In [ ]:
train=files.upload()
tst=files.upload()

Saving train.csv to train.csv


Saving test.csv to test.csv


In [ ]:
tr = pd.read_csv(io.BytesIO(train['train.csv']))
tst = pd.read_csv(io.BytesIO(tst['test.csv']))

tr.head()
tst.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [ ]:
tst.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


데이터에 있는 사람들의 정보를 이용해 그 사람의 구조 여부를 예측하는 것이 이번 문제이다.

우리는 오로지 수치 데이터만 필요하기 때문에 Name 과 Ticket 항목은 `drop` 메소드를 통해 제외시키자. PassengerId와 선실 번호를 의미하는 Cabin도 유의미한 항목으로 보여지지 않기 때문에 제외시키자.

In [ ]:
tr = tr.drop(["PassengerId", "Name", "Ticket", "Cabin"], axis=1)
tst = tst.drop(["PassengerId", "Name", "Ticket", "Cabin"], axis=1)

tr.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


이 후 비어있는 데이터가 있는지 `isna` 메소드로 확인하자. 데이터가 비어있으면 모델 적용 할 때 오류가 날 것이다.

In [ ]:
tr.isna().sum()

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Embarked      2
dtype: int64

In [ ]:
tst.isna().sum()

Pclass       0
Sex          0
Age         86
SibSp        0
Parch        0
Fare         1
Embarked     0
dtype: int64

비어있는 데이터 처리 방법은 여러가지가 있다. 가장 쉬운 방법은 하나라도 비어있는 데이터가 있으면 그 행을 지워버리는 것이다. 그러나 실전에서는 데이터 하나가 아까운 경우가 많다. 따라서 이번에는 적절한 값으로 빈 데이터를 채워넣는 방법을 이용할 것이다. 가장 간단한 방법으로는 해당 요소에 대한 평균값으로 채워 넣는 것이다. 이 외에도 거리함수를 이용해 가장 가까운 데이터의 요소 값으로 채워넣는 등, 여러가지 방법이 있겠지만 일단 가장 간단한 방법으로 진행 해보자. 결측값을 채우는 것은 `fiilna` 메소드로 할 수 있다.

In [ ]:
meanAge_tr = tr.mean()["Age"]
meanAge_tst = tst.mean()["Age"]
fare_tst = tst.mean()["Fare"]

meanAge_tr, meanAge_tst, fare_tst

(29.69911764705882, 30.272590361445783, 35.6271884892086)

In [ ]:
tr["Age"] = tr["Age"].fillna(meanAge_tr)
tst["Age"] = tst["Age"].fillna(meanAge_tst)
tst["Fare"] = tst["Fare"].fillna(fare_tst)

Embarked 는 범주형 데이터이기 때문에 평균값 대신 최빈값으로 채울 수 있다.

In [ ]:
embarked_tr = tr.mode()["Embarked"]

embarked_tr

0    S
Name: Embarked, dtype: object

In [ ]:
tr["Embarked"] = tr["Embarked"].fillna("S")

In [ ]:
tr.isna().sum()

Survived    0
Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    0
dtype: int64

In [ ]:
tst.isna().sum()

Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    0
dtype: int64

In [ ]:
tr.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


범주형 데이터인 Pclass, Sex, Embarked 를 수치로 바꾸자. Sex와 Embarked 는 문자열이기 때문에 수치로 바꿀 필요가 있지만, Pclass 의 값은 1, 2, 3이기 때문에 1만 빼주면 된다. 데이터를 특정 값으로 변환하는 것은 `apply` 메소드를 통해 실행 할 수 있다.

In [ ]:
map_sex = { "male": 0, "female": 1 }
map_embarked = { "C": 0, "Q": 1, "S": 2 }

for df in [tr, tst]:
    df["Pclass"] = df["Pclass"].apply(lambda x: x - 1)
    df["Sex"] = df["Sex"].apply(lambda x: map_sex[x])
    df["Embarked"] = df["Embarked"].apply(lambda x: map_embarked[x])
    
tr.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,2,0,22.0,1,0,7.2500,2
1,1,0,1,38.0,1,0,71.2833,0
2,1,2,1,26.0,0,0,7.9250,2
3,1,0,1,35.0,1,0,53.1000,2
4,0,2,0,35.0,0,0,8.0500,2


In [ ]:
tr.dtypes

Survived      int64
Pclass        int64
Sex           int64
Age         float64
SibSp         int64
Parch         int64
Fare        float64
Embarked      int64
dtype: object

전부 수치 데이터로 바뀐 것을 알 수 있다. 이제 범주형 데이터를 one-hot encoding 해서 추가하자.

In [ ]:
cols = ["Pclass", "Sex", "Embarked"]
counts = [3, 2, 3] # 범주 개수

tr_ = pd.DataFrame(tr)
tst_ = pd.DataFrame(tst)

for c, count in zip(cols, counts):
    for df in [tr_, tst_]:
        n = df.shape[0]
        arr = np.zeros((n, count))
        arr[np.arange(n), df[c]] = 1
        for i in range(count):
            df[c + "_" + str(i)] = arr[:, i]
            
tr_.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Pclass_0,Pclass_1,Pclass_2,Sex_0,Sex_1,Embarked_0,Embarked_1,Embarked_2
0,0,2,0,22.0,1,0,7.2500,2,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0
1,1,0,1,38.0,1,0,71.2833,0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
2,1,2,1,26.0,0,0,7.9250,2,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
3,1,0,1,35.0,1,0,53.1000,2,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
4,0,2,0,35.0,0,0,8.0500,2,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0


범주형 데이터에 대한 정보는 one-hot encoding 이 된 열에 있기 때문에 원래 데이터는 삭제를 해주어도 된다.

Age, Fare는 다른 데이터와 단위가 다르기 때문에 표준화 해 주자.

In [ ]:
for df in [tr_, tst_]:
    for c in ["Age", "Fare"]:
        df[c] = (df[c] - df[c].mean()) / df[c].std()
    df = df.drop(["Pclass", "Sex", "Embarked"], axis=1)
    
tr_.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Pclass_0,Pclass_1,Pclass_2,Sex_0,Sex_1,Embarked_0,Embarked_1,Embarked_2
0,0,2,0,-0.592148,1,0,-0.502163,2,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0
1,1,0,1,0.638430,1,0,0.786404,0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
2,1,2,1,-0.284503,0,0,-0.488580,2,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
3,1,0,1,0.407697,1,0,0.420494,2,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
4,0,2,0,0.407697,0,0,-0.486064,2,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0


다중공선성 확인을 위해 상관계수를 구해보자. 만약 상관관계가 높은 열이 있다면 삭제를 고려 할 수 있다. 상관계수의 절댓값이 0.5보다 높은 경우를 출력 해보자.

In [ ]:
tr_.corr()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Pclass_0,Pclass_1,Pclass_2,Sex_0,Sex_1,Embarked_0,Embarked_1,Embarked_2
Survived,1.000000,-0.338481,0.543351,-0.069809,-0.035322,0.081629,0.257307,-0.167675,0.285904,0.093349,-0.322308,-0.543351,0.543351,0.168240,0.003650,-0.149683
Pclass,-0.338481,1.000000,-0.131900,-0.331339,0.083081,0.018443,-0.549500,0.162098,-0.885924,-0.188432,0.916673,0.131900,-0.131900,-0.243292,0.221009,0.074053
Sex,0.543351,-0.131900,1.000000,-0.084153,0.114631,0.245489,0.182333,-0.108262,0.098013,0.064746,-0.137143,-1.000000,1.000000,0.082853,0.074115,-0.119224
Age,-0.069809,-0.331339,-0.084153,1.000000,-0.232625,-0.179191,0.091566,-0.026749,0.319916,0.006589,-0.281004,0.084153,-0.084153,0.032024,-0.013855,-0.019336
SibSp,-0.035322,0.083081,0.114631,-0.232625,1.000000,0.414838,0.159651,0.068230,-0.054582,-0.055932,0.092548,-0.114631,0.114631,-0.059528,-0.026354,0.068734
Parch,0.081629,0.018443,0.245489,-0.179191,0.414838,1.000000,0.216225,0.039798,-0.017633,-0.000734,0.015790,-0.245489,0.245489,-0.011069,-0.081228,0.060814
Fare,0.257307,-0.549500,0.182333,0.091566,0.159651,0.216225,1.000000,-0.224719,0.591711,-0.118557,-0.413333,-0.182333,0.182333,0.269335,-0.117216,-0.162184
Embarked,-0.167675,0.162098,-0.108262,-0.026749,0.068230,0.039798,-0.224719,1.000000,-0.237965,0.169245,0.067291,0.108262,-0.108262,-0.936273,-0.208581,0.951474
Pclass_0,0.285904,-0.885924,0.098013,0.319916,-0.054582,-0.017633,0.591711,-0.237965,1.000000,-0.288585,-0.626738,-0.098013,0.098013,0.296423,-0.155342,-0.161921
Pclass_1,0.093349,-0.188432,0.064746,0.006589,-0.055932,-0.000734,-0.118557,0.169245,-0.288585,1.000000,-0.565210,-0.064746,0.064746,-0.125416,-0.127301,0.189980


In [ ]:
list(enumerate(tr_.columns))

[(0, 'Survived'),
 (1, 'Pclass'),
 (2, 'Sex'),
 (3, 'Age'),
 (4, 'SibSp'),
 (5, 'Parch'),
 (6, 'Fare'),
 (7, 'Embarked'),
 (8, 'Pclass_0'),
 (9, 'Pclass_1'),
 (10, 'Pclass_2'),
 (11, 'Sex_0'),
 (12, 'Sex_1'),
 (13, 'Embarked_0'),
 (14, 'Embarked_1'),
 (15, 'Embarked_2')]

In [ ]:
for n, i in enumerate(tr_.columns):
    for j in tr_.columns[n + 1:]:
        if i == j or  i == "Survived" or j == "Survived":
            continue
        if abs(tr_.corr()[i][j]) > 0.5:
            print(i, j, tr_.corr()[i][j])

Pclass Fare -0.5494996199439068
Pclass Pclass_0 -0.8859238618875467
Pclass Pclass_2 0.9166727522685145
Sex Sex_0 -1.0
Sex Sex_1 1.0
Fare Pclass_0 0.5917107188835091
Embarked Embarked_0 -0.9362730393120936
Embarked Embarked_2 0.9514738260590868
Pclass_0 Pclass_2 -0.6267375846396964
Pclass_1 Pclass_2 -0.5652101535538238
Sex_0 Sex_1 -1.0
Embarked_0 Embarked_2 -0.7827421287187183


Fare 가 Pclass_0 과 높은 상관관계를 가지고 있음을 볼 수 있다. 따라서 Fare 를 삭제 해주자.
- 사실 다중공선성 문제에 대처 할 때에는, 위와 같이 무작정 삭제하는 방법을 좋지 않고, 실제 데이터를 시각화 하여 판단하거나, 도메인 지식을 근거로 삭제하는 등 더욱 심도 있는 작업이 필요하다.

In [ ]:
tr_ = tr_.drop(["Fare"], axis=1)
tst_ = tst_.drop(["Fare"], axis=1)

데이터 전처리 작업을 마치고 모델을 선택 해 보자. 먼저 대표적인 분류 모델인 로지스틱 회귀모델을 이용 해보자.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [ ]:
X = np.array(tr_.drop(["Survived"], axis=1))
y = np.array(tr_["Survived"])

X.shape

(891, 14)

In [ ]:
X_tr, X_tst, y_tr, y_tst = train_test_split(X, y, test_size=0.3)

X_tr.shape, X_tst.shape

((623, 14), (268, 14))

In [ ]:
model = LogisticRegression()
model.fit(X_tr, y_tr)

y_hat = model.predict(X_tst)


np.sum(y_tst==y_hat) / y_hat.shape[0]

array([False,  True,  True, False,  True, False, False, False,  True,
        True, False,  True,  True,  True,  True,  True, False,  True,
        True,  True,  True,  True, False,  True,  True, False,  True,
        True,  True,  True, False,  True,  True,  True,  True, False,
        True,  True,  True,  True,  True,  True,  True, False,  True,
        True,  True, False, False,  True, False,  True,  True,  True,
       False,  True,  True,  True,  True, False,  True,  True,  True,
       False,  True,  True,  True, False, False,  True,  True,  True,
        True, False,  True,  True,  True,  True,  True, False, False,
       False, False, False,  True,  True,  True,  True,  True, False,
        True,  True,  True,  True,  True,  True, False, False, False,
        True,  True, False,  True,  True,  True, False,  True,  True,
        True,  True, False, False,  True, False,  True,  True,  True,
       False,  True, False, False,  True,  True, False, False,  True,
        True,  True,

다음으로 knn 모델을 써보자.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
for k in range(1, 25, 2):
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_tr, y_tr)
    y_hat = knn.predict(X_tst)
    
    print(k, np.sum(y_tst==y_hat) / y_tst.shape[0])

1 0.7052238805970149
3 0.7761194029850746
5 0.7910447761194029
7 0.7835820895522388
9 0.7873134328358209
11 0.7686567164179104
13 0.7649253731343284
15 0.7686567164179104
17 0.7649253731343284
19 0.8059701492537313
21 0.8134328358208955
23 0.8059701492537313


K=11 일 때의 모델이 가장 좋은 성능을 보인다.

두 모델을 합쳐서 테스트 해보자. 즉, 두 모델이 출력하는 확률의 평균을 이용 해 보자.

In [ ]:
knn = KNeighborsClassifier(n_neighbors=11)
knn.fit(X_tr, y_tr)

log_prob = model.predict_proba(X_tst)
knn_prob = knn.predict_proba(X_tst)

y_prob = (log_prob + knn_prob) / 2
y_hat = np.argmax(y_prob, axis=1)

np.sum(y_tst==y_hat) / y_tst.shape[0]

0.7649253731343284

정답 파일을 만들어 제출한다.

In [ ]:
tmp=files.upload()
tst_original = pd.read_csv(io.BytesIO(tmp['test.csv']))
ans = pd.DataFrame()

log_prob = model.predict_proba(np.array(tst_))
knn_prob = knn.predict_proba(np.array(tst_))
y_prob = (log_prob + knn_prob) / 2
y_hat = np.argmax(y_prob, axis=1)

ans["PassengerID"] = tst_original["PassengerId"]
ans["Survived"] = y_hat

ans.head()

Saving test.csv to test (1).csv


NameError: ignored

In [ ]:
ans.to_csv("a.csv")
files.download("a.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

위 모델을 0.77990 점을 받았다. 더 높은 점수를 받을 수 있는 방법을 생각해보자.